Data Selection Impact on Model Variability
T-tests

In [ ]:
import os
import pandas as pd
from scipy.stats import ttest_rel
import numpy as np
def t_test(original, selection):
    """Comparing method"""
    def two_tailed_t_test(original, selection):
        n_d = len(selection)
        n_c = len(original)
        n = min(n_d, n_c)
        t, p = ttest_rel(original[:n], selection[:n])
        if np.isnan(t):
            t, p = 0, 1
        return {"t-stats":t, "p-value":p}

    def one_tailed_t_test(original, selection, direction):
        two_tail = two_tailed_t_test(original, selection)
        t, p_two = two_tail['t-stats'], two_tail['p-value']
        if direction == 'positive':
            if t > 0 :
                p = p_two * 0.5
            else:
                p = 1 - p_two * 0.5
        else:
            if t < 0:
                p = p_two * 0.5
            else:
                p = 1 - p_two * 0.5
        return {"t-stats":t, "p-value":p}

    result = {}
    result['two_tail'] = two_tailed_t_test(original, selection)
    result['one_tail_pos'] = one_tailed_t_test(original, selection, 'positive')
    result['one_tail_neg'] = one_tailed_t_test(original, selection, 'negative')
    return result


def evaluate_score(original, selection):
    alpha =  0.05
    results = t_test(original, selection)
    difference = 'insignificant'

    if results['two_tail']['p-value'] < alpha:
        if results['one_tail_neg']['p-value'] < alpha:
            difference = 'positive'
        if results['one_tail_pos']['p-value'] < alpha:
            difference = 'negative'

    return difference


T-tests

ARS dataset

In [ ]:
import os
import pandas as pd
import numpy as np
# Define paths and initialize variables
systems_path = ['../results/ars-selection/CRAIGPB', '../results/ars-selection/GLISTERPB', '../results/ars-selection/GradMatchPB', '../results/ars-selection/Random']
ratio_path = ['/ars_0.05', '/ars_0.1', '/ars_0.2', '/ars_0.3', '/ars_0.5']
models = ['Logreg', 'MLP', 'SVM']

directory_full = '../results/ars-selection/Full/ars_1'
# Metrics tables
index_labels = ['negative-f', 'insignificant-f','positive-f','negative-u', 'insignificant-u','positive-u','negative-c', 'insignificant-c','positive-c', 'insignificant-f-u and positive-c' ]
# Initialize influence table
cols = ['ratio', 'system', 'time', 'acc', 'F1_score', 'Precision', 'Recall', 'SPD_gender', 'EOD_gender', 'AOD_gender', 'DI_gender', 'DcI_gender']
compute_influence = pd.DataFrame(columns=cols)


# Initialize full result and cost DataFrames


excluded_columns = ['DI_gender', 'DI_age', 'DI_race']

i = 0
files = []

df_average_results = pd.DataFrame()

# Load full system results for comparisons
for model in models: 


    df_full_result_avg = pd.DataFrame()
    df_full_cost_avg = pd.DataFrame()
    df_full_cost_time_avg = pd.DataFrame()

    df_full_result_std = pd.DataFrame()
    df_full_cost_std = pd.DataFrame()
    df_full_cost_time_std = pd.DataFrame()

    df_full_result_vc = pd.DataFrame()
    df_full_cost_vc = pd.DataFrame()
    df_full_cost_time_vc = pd.DataFrame()

    for filename in os.listdir(directory_full):
        if "fair_metrics_"+ model in filename :
           file_path = os.path.join(directory_full, filename)
           if model == 'SVM':
               df_r = pd.read_csv(file_path).iloc[34:69, 1:]
           else:
               df_r = pd.read_csv(file_path).iloc[54:69, 1:]
           df_r.update(df_r.drop(columns=excluded_columns, errors='ignore') * 100)

           df_r_vc = df_r.abs().std().to_frame().T / df_r.abs().mean()

           df_full_result_avg = pd.concat([df_full_result_avg, df_r.abs().mean().to_frame().T], ignore_index=True)
           df_full_result_std = pd.concat([df_full_result_std, df_r.abs().std().to_frame().T], ignore_index=True)
           df_full_result_vc = pd.concat([df_full_result_vc, df_r_vc], ignore_index=True)

        if "cost_metrics_" + model in filename and filename.endswith("_0.csv"):
            file_path = os.path.join(directory_full, filename)
            if model == 'SVM':
                df_ct = pd.read_csv(file_path).iloc[34:69, 1:]  
            else:
                df_ct = pd.read_csv(file_path).iloc[54:69, 1:]  

            
            df_ct_vc = df_ct.abs().std().to_frame().T / df_ct.abs().mean()

            
            df_full_cost_time_avg = pd.concat([df_full_cost_time_avg, df_ct.abs().mean().to_frame().T], ignore_index=True)
            df_full_cost_time_std = pd.concat([df_full_cost_time_std, df_ct.abs().std().to_frame().T], ignore_index=True)
            df_full_cost_time_vc = pd.concat([df_full_cost_time_vc, df_ct_vc], ignore_index=True)

        if "cost_metrics_" + model in filename :
            file_path = os.path.join(directory_full, filename)
            if model == 'SVM':
                df_c = pd.read_csv(file_path).iloc[34:69, 1:]
            else:
                df_c = pd.read_csv(file_path).iloc[54:69, 1:]

            df_c_vc = df_c.abs().std().to_frame().T / df_c.abs().mean()
            
            df_full_cost_avg = pd.concat([df_full_cost_avg, df_c.abs().mean().to_frame().T], ignore_index=True)
            df_full_cost_std = pd.concat([df_full_cost_std, df_c.abs().std().to_frame().T], ignore_index=True)
            df_full_cost_vc = pd.concat([df_full_cost_vc, df_c_vc], ignore_index=True)

    print('**********************')
    print(model)


    #for col in ['DI_gender']:
    #    df_full_result[col] = df_full_result[col].apply(lambda x: abs(1 - x))
    

    ### Calcul de valeurs moyenne pour le full 
    result_avg_avg = df_full_result_avg.mean()
    cost_avg_avg = df_full_cost_avg.mean()
    cost_time_avg_avg = df_full_cost_time_avg.mean()

    result_std_avg = df_full_result_std.mean()
    cost_std_avg = df_full_cost_std.mean()
    cost_time_std_avg = df_full_cost_time_std.mean()

    result_vc_avg = df_full_result_vc.mean()
    cost_vc_avg = df_full_cost_vc.mean()
    cost_time_vc_avg = df_full_cost_time_vc.mean()

   

    # Ligne de valeurs moyennes de full 
    avg_data = pd.DataFrame({
        **{str(col) + '_avg_avg': [result_avg_avg[col]] for col in result_avg_avg.index},
        **{str(col) + '_avg_avg': [cost_avg_avg[col]] for col in cost_avg_avg.index},
        **{str(col) + '_avg_avg': [cost_time_avg_avg[col]] for col in cost_time_avg_avg.index},
        **{str(col) + '_std_avg': [result_std_avg[col]] for col in result_std_avg.index},
        **{str(col) + '_std_avg': [cost_std_avg[col]] for col in cost_std_avg.index},
        **{str(col) + '_std_avg': [cost_time_std_avg[col]] for col in cost_time_std_avg.index},
        **{str(col) + '_vc_avg': [result_vc_avg[col]] for col in result_vc_avg.index},
        **{str(col) + '_vc_avg': [cost_vc_avg[col]] for col in cost_vc_avg.index},
        **{str(col) + '_vc_avg': [cost_time_vc_avg[col]] for col in cost_time_vc_avg.index},
        'dataset': ['ars'],
        'model': [model],
        'system': ['Full'],
        'ratio': [1],
    })

    df_average_results = pd.concat([df_average_results, avg_data], ignore_index=True)

    # Lire les fichiers de système de selection par système et par ratio

    for directory_path_1 in systems_path:
       for directory_path_2 in ratio_path:
           directory = directory_path_1 + directory_path_2
           system = directory_path_1.split('/')[-1]           
        
           df_full_result_selection_avg = pd.DataFrame()
           df_full_cost_selection_avg = pd.DataFrame()
           df_full_cost_time_selection_avg = pd.DataFrame()

           df_full_result_selection_std = pd.DataFrame()
           df_full_cost_selection_std = pd.DataFrame()
           df_full_cost_time_selection_std = pd.DataFrame()

           df_full_result_selection_vc = pd.DataFrame()
           df_full_cost_selection_vc = pd.DataFrame()
           df_full_cost_time_selection_vc = pd.DataFrame()
        
           # Cost file processing
           for filename in os.listdir(directory):
                if "fair_metrics_" + model in filename:
                    file_path = os.path.join(directory, filename)
                    if model == 'SVM':
                        df_to_add_f = pd.read_csv(file_path).iloc[14:49, 1:].abs()
                    else:
                        df_to_add_f = pd.read_csv(file_path).iloc[34:49, 1:].abs()
                    df_to_add_f.update(df_to_add_f.drop(columns=excluded_columns, errors='ignore') * 100)

                    df_to_add_f_vc = df_to_add_f.abs().std().to_frame().T / df_to_add_f.abs().mean()

                    df_full_result_selection_avg = pd.concat([df_full_result_selection_avg, df_to_add_f.mean().to_frame().T], ignore_index=True)
                    df_full_result_selection_std = pd.concat([df_full_result_selection_std, df_to_add_f.std().to_frame().T], ignore_index=True)
                    df_full_result_selection_vc = pd.concat([df_full_result_selection_vc, df_to_add_f_vc], ignore_index=True)
                    
                if (("cost_metrics_" + model in filename) and (filename.endswith("_0.csv"))):
                    file_path = os.path.join(directory, filename)
                    if model == 'SVM':
                        df_to_add_t = pd.read_csv(file_path).iloc[14:49, 1:].abs()
                    else:
                        df_to_add_t = pd.read_csv(file_path).iloc[34:49, 1:].abs()

                    df_to_add_t_vc = df_to_add_t.abs().std().to_frame().T / df_to_add_t.abs().mean()
                    
                    df_full_cost_time_selection_avg = pd.concat([df_full_cost_time_selection_avg, df_to_add_t.mean().to_frame().T], ignore_index=True)
                    df_full_cost_time_selection_std = pd.concat([df_full_cost_time_selection_std, df_to_add_t.std().to_frame().T], ignore_index=True)
                    df_full_cost_time_selection_vc = pd.concat([df_full_cost_time_selection_vc, df_to_add_t_vc], ignore_index=True)
        
                if "cost_metrics_" + model in filename:
                    file_path = os.path.join(directory, filename)
                    if model == 'SVM':
                        df_to_add_c = pd.read_csv(file_path).iloc[14:49, 1:].abs()
                    else:
                        df_to_add_c = pd.read_csv(file_path).iloc[34:49, 1:].abs()

                    
                    df_to_add_c_vc = df_to_add_c.abs().std().to_frame().T / df_to_add_c.abs().mean()
                    
                    df_full_cost_selection_avg = pd.concat([df_full_cost_selection_avg, df_to_add_c.mean().to_frame().T], ignore_index=True)
                    df_full_cost_selection_std = pd.concat([df_full_cost_selection_std, df_to_add_c.std().to_frame().T], ignore_index=True)
                    df_full_cost_selection_vc = pd.concat([df_full_cost_selection_vc, df_to_add_c_vc], ignore_index=True)
           
                
           #for col in ['DI_gender']:
           #  df_full_result_selection[col] = df_full_result_selection[col].apply(lambda x: abs(1 - x))
           
           ### Calcul de valeurs moyenne pour le full 
           result_sel_avg_avg = df_full_result_selection_avg.mean()
           cost_sel_avg_avg = df_full_cost_selection_avg.mean()
           cost_time_sel_avg_avg = df_full_cost_time_selection_avg.mean()

           result_sel_std_avg = df_full_result_selection_std.mean()
           cost_sel_std_avg = df_full_cost_selection_std.mean()
           cost_time_sel_std_avg = df_full_cost_time_selection_std.mean()

           result_sel_vc_avg = df_full_result_selection_vc.mean()
           cost_sel_vc_avg = df_full_cost_selection_vc.mean()
           cost_time_sel_vc_avg = df_full_cost_time_selection_vc.mean()

           ## Faire les t-test
           # Cost
           # time
           test_time = evaluate_score(df_full_cost_time_selection_vc['Full_training_time'].to_numpy(), df_full_cost_time_vc['Full_training_time'].to_numpy()) 
           
           # Model quality
           # accuracy
           test_acc = evaluate_score(df_full_cost_selection_vc['accuracy'].to_numpy(), df_full_cost_vc['accuracy'].to_numpy()) 
           # F1-score
           test_f1 = evaluate_score(df_full_result_selection_vc['F1_score'].to_numpy(), df_full_result_vc['F1_score'].to_numpy()) 
           #precision
           test_precision = evaluate_score(df_full_result_selection_vc['Precision'].to_numpy(), df_full_result_vc['Precision'].to_numpy()) 
           # recall 
           test_recall = evaluate_score(df_full_result_selection_vc['Recall'].to_numpy(), df_full_result_vc['Recall'].to_numpy()) 

           # Fairness
           # SPD gender
           test_spd_g = evaluate_score(df_full_result_selection_vc['SPD_gender'].to_numpy(), df_full_result_vc['SPD_gender'].to_numpy())
           # EOD gender
           test_eod_g = evaluate_score(df_full_result_selection_vc['EOD_gender'].to_numpy(), df_full_result_vc['EOD_gender'].to_numpy())
           # AOD gender
           test_aod_g = evaluate_score(df_full_result_selection_vc['AOD_gender'].to_numpy(), df_full_result_vc['AOD_gender'].to_numpy())
           # DI gender
           test_di_g = evaluate_score(df_full_result_selection_vc['DI_gender'].to_numpy(), df_full_result_vc['DI_gender'].to_numpy())
           # DcI gender
           test_dci_g = evaluate_score(df_full_result_selection_vc['DcI_gender'].to_numpy(), df_full_result_vc['DcI_gender'].to_numpy())


           
           avg_sel_data = pd.DataFrame({
            **{str(col) + '_avg_avg': [result_sel_avg_avg[col]] for col in result_sel_avg_avg.index},
            **{str(col) + '_avg_avg': [cost_sel_avg_avg[col]] for col in cost_sel_avg_avg.index},
            **{str(col) + '_avg_avg': [cost_time_sel_avg_avg[col]] for col in cost_time_sel_avg_avg.index},
            **{str(col) + '_std_avg': [result_sel_std_avg[col]] for col in result_sel_std_avg.index},
            **{str(col) + '_std_avg': [cost_sel_std_avg[col]] for col in cost_sel_std_avg.index},
            **{str(col) + '_std_avg': [cost_time_sel_std_avg[col]] for col in cost_time_sel_std_avg.index},
            **{str(col) + '_vc_avg': [result_sel_vc_avg[col]] for col in result_sel_vc_avg.index},
            **{str(col) + '_vc_avg': [cost_sel_vc_avg[col]] for col in cost_sel_vc_avg.index},
            **{str(col) + '_vc_avg': [cost_time_sel_vc_avg[col]] for col in cost_time_sel_vc_avg.index},
           'dataset': ['ars'],
           'model': [model],
           'system': [system],
           'ratio': [directory_path_2.split('_')[-1]],
           'test_time' : [test_time], 
           'test_acc' : [test_acc], 
           'test_f1' : [test_f1], 
           'test_precision' : [test_precision], 
           'test_recall' : [test_recall],
           'test_SPD_gender' : [test_spd_g], 
           'test_EOD_gender' : [test_eod_g], 
           'test_AOD_gender' : [test_aod_g], 
           'test_DI_gender' : [test_di_g], 
           'test_DcI_gender' : [test_dci_g], 

           'test_SPD_age' : ' ',
           'test_EOD_age' : ' ',
           'test_AOD_age' : ' ', 
           'test_DI_age' : ' ', 
           'test_DcI_age' : ' ', 

           'test_SPD_race' : ' ',
           'test_EOD_race' : ' ',
           'test_AOD_race' : ' ', 
           'test_DI_race' : ' ', 
           'test_DcI_race' : ' ', 
            })
           
           df_average_results = pd.concat([df_average_results, avg_sel_data], ignore_index=True)



df_average_results.to_csv('../results/test/ars_ttest_results_epochs_intervals_vc_metrics.csv', index=False)



Adult dataset

In [ ]:
import os
import pandas as pd
import numpy as np
# Define paths and initialize variables
systems_path = ['../results/adult-selection/CRAIGPB', '../results/adult-selection/GLISTERPB', '../results/adult-selection/GradMatchPB', '../results/adult-selection/Random']
ratio_path = ['/adult_0.05', '/adult_0.1', '/adult_0.2','/adult_0.3', '/adult_0.5']
models = ['Logreg', 'MLP', 'SVM']  

directory_full = '../results/adult-selection/Full/adult_1'




excluded_columns = ['DI_gender', 'DI_age', 'DI_race']

i = 0
files = []

df_average_results = pd.DataFrame()

# Load full system results for comparisons
for model in models: 

    df_full_result_avg = pd.DataFrame()
    df_full_cost_avg = pd.DataFrame()
    df_full_cost_time_avg = pd.DataFrame()

    df_full_result_std = pd.DataFrame()
    df_full_cost_std = pd.DataFrame()
    df_full_cost_time_std = pd.DataFrame()

    df_full_result_vc = pd.DataFrame()
    df_full_cost_vc = pd.DataFrame()
    df_full_cost_time_vc = pd.DataFrame()

    for filename in os.listdir(directory_full):
        if "fair_metrics_"+ model in filename :
           file_path = os.path.join(directory_full, filename)
           if model == 'MLP':
             df_r = pd.read_csv(file_path).iloc[89:199, 1:]
           else :
             df_r = pd.read_csv(file_path).iloc[54:99, 1:]
           df_r.update(df_r.drop(columns=excluded_columns, errors='ignore') * 100)
           df_r_vc = df_r.abs().std().to_frame().T / df_r.abs().mean()

           df_full_result_avg = pd.concat([df_full_result_avg, df_r.abs().mean().to_frame().T], ignore_index=True)
           df_full_result_std = pd.concat([df_full_result_std, df_r.abs().std().to_frame().T], ignore_index=True)
           df_full_result_vc = pd.concat([df_full_result_vc, df_r_vc], ignore_index=True)

        if "cost_metrics_" + model in filename and filename.endswith("_0.csv"):
            file_path = os.path.join(directory_full, filename)
            if model == 'MLP':
             df_ct = pd.read_csv(file_path).iloc[89:199, 1:]
            else :
             df_ct = pd.read_csv(file_path).iloc[54:99, 1:]
            df_ct_vc = df_ct.abs().std().to_frame().T / df_ct.abs().mean()
            
            df_full_cost_time_avg = pd.concat([df_full_cost_time_avg, df_ct.abs().mean().to_frame().T], ignore_index=True)
            df_full_cost_time_std = pd.concat([df_full_cost_time_std, df_ct.abs().std().to_frame().T], ignore_index=True)
            df_full_cost_time_vc = pd.concat([df_full_cost_time_vc, df_ct_vc], ignore_index=True)

        if "cost_metrics_" + model in filename :
            file_path = os.path.join(directory_full, filename)
            if model == 'MLP':
             df_c = pd.read_csv(file_path).iloc[89:199, 1:]
            else :
             df_c = pd.read_csv(file_path).iloc[54:99, 1:]

            df_c_vc = df_c.abs().std().to_frame().T / df_c.abs().mean()

            df_full_cost_avg = pd.concat([df_full_cost_avg, df_c.abs().mean().to_frame().T], ignore_index=True)
            df_full_cost_std = pd.concat([df_full_cost_std, df_c.abs().std().to_frame().T], ignore_index=True)
            df_full_cost_vc = pd.concat([df_full_cost_vc, df_c_vc], ignore_index=True)

    print('**********************')
    print(model)

    #for col in ['DI_gender', 'DI_age', 'DI_race']:
    #    df_full_result[col] = df_full_result[col].apply(lambda x: abs(1 - x))
    

    ### Calcul de valeurs moyenne pour le full 
    result_avg_avg = df_full_result_avg.mean()
    cost_avg_avg = df_full_cost_avg.mean()
    cost_time_avg_avg = df_full_cost_time_avg.mean()

    result_std_avg = df_full_result_std.mean()
    cost_std_avg = df_full_cost_std.mean()
    cost_time_std_avg = df_full_cost_time_std.mean()

    result_vc_avg = df_full_result_vc.mean()
    cost_vc_avg = df_full_cost_vc.mean()
    cost_time_vc_avg = df_full_cost_time_vc.mean()

   

    # Ligne de valeurs moyennes de full 
    avg_data = pd.DataFrame({
        **{str(col) + '_avg_avg': [result_avg_avg[col]] for col in result_avg_avg.index},
        **{str(col) + '_avg_avg': [cost_avg_avg[col]] for col in cost_avg_avg.index},
        **{str(col) + '_avg_avg': [cost_time_avg_avg[col]] for col in cost_time_avg_avg.index},
        **{str(col) + '_std_avg': [result_std_avg[col]] for col in result_std_avg.index},
        **{str(col) + '_std_avg': [cost_std_avg[col]] for col in cost_std_avg.index},
        **{str(col) + '_std_avg': [cost_time_std_avg[col]] for col in cost_time_std_avg.index},
        **{str(col) + '_vc_avg': [result_vc_avg[col]] for col in result_vc_avg.index},
        **{str(col) + '_vc_avg': [cost_vc_avg[col]] for col in cost_vc_avg.index},
        **{str(col) + '_vc_avg': [cost_time_vc_avg[col]] for col in cost_time_vc_avg.index},
        'dataset': ['adult'],
        'model': [model],
        'system': ['Full'],
        'ratio': [1],
    })

    df_average_results = pd.concat([df_average_results, avg_data], ignore_index=True)

    # Lire les fichiers de système de selection par système et par ratio

    for directory_path_1 in systems_path:
       for directory_path_2 in ratio_path:
           directory = directory_path_1 + directory_path_2
           system = directory_path_1.split('/')[-1]           
        
           df_full_result_selection_avg = pd.DataFrame()
           df_full_cost_selection_avg = pd.DataFrame()
           df_full_cost_time_selection_avg = pd.DataFrame()

           df_full_result_selection_std = pd.DataFrame()
           df_full_cost_selection_std = pd.DataFrame()
           df_full_cost_time_selection_std = pd.DataFrame()

           df_full_result_selection_vc = pd.DataFrame()
           df_full_cost_selection_vc = pd.DataFrame()
           df_full_cost_time_selection_vc = pd.DataFrame()
        
           # Cost file processing
           for filename in os.listdir(directory):
                if "fair_metrics_" + model in filename:
                    file_path = os.path.join(directory, filename)
                    if model == 'MLP':
                       df_to_add_f = pd.read_csv(file_path).iloc[69:179, 1:].abs()
                       
                    else:
                       df_to_add_f = pd.read_csv(file_path).iloc[34:79, 1:].abs()
                    df_to_add_f.update(df_to_add_f.drop(columns=excluded_columns, errors='ignore') * 100)

                    df_to_add_f_vc = df_to_add_f.abs().std().to_frame().T / df_to_add_f.abs().mean()
                    
                    df_full_result_selection_avg = pd.concat([df_full_result_selection_avg, df_to_add_f.mean().to_frame().T], ignore_index=True)
                    df_full_result_selection_std = pd.concat([df_full_result_selection_std, df_to_add_f.std().to_frame().T], ignore_index=True)
                    df_full_result_selection_vc = pd.concat([df_full_result_selection_vc, df_to_add_f_vc], ignore_index=True)
                    

                if (("cost_metrics_" + model in filename) and (filename.endswith("_0.csv"))):
                    file_path = os.path.join(directory, filename)
                    if model == 'MLP':
                       df_to_add_t = pd.read_csv(file_path).iloc[69:179, 1:].abs()
                    else:
                       df_to_add_t = pd.read_csv(file_path).iloc[34:79, 1:].abs()
                    
                    df_to_add_t_vc = df_to_add_t.abs().std().to_frame().T / df_to_add_t.abs().mean()
                    
                    df_full_cost_time_selection_avg = pd.concat([df_full_cost_time_selection_avg, df_to_add_t.mean().to_frame().T], ignore_index=True)
                    df_full_cost_time_selection_std = pd.concat([df_full_cost_time_selection_std, df_to_add_t.std().to_frame().T], ignore_index=True)
                    df_full_cost_time_selection_vc = pd.concat([df_full_cost_time_selection_vc, df_to_add_t_vc], ignore_index=True)
    
    
    

        
                if "cost_metrics_" + model in filename:
                    file_path = os.path.join(directory, filename)
                    if model == 'MLP':
                       df_to_add_c = pd.read_csv(file_path).iloc[69:179, 1:].abs()
                    else:
                       df_to_add_c = pd.read_csv(file_path).iloc[34:79, 1:].abs()
                    
                    df_to_add_c_vc = df_to_add_c.abs().std().to_frame().T / df_to_add_c.abs().mean()
                    
                    df_full_cost_selection_avg = pd.concat([df_full_cost_selection_avg, df_to_add_c.mean().to_frame().T], ignore_index=True)
                    df_full_cost_selection_std = pd.concat([df_full_cost_selection_std, df_to_add_c.std().to_frame().T], ignore_index=True)
                    df_full_cost_selection_vc = pd.concat([df_full_cost_selection_vc, df_to_add_c_vc], ignore_index=True)
           
                
           #for col in ['DI_gender', 'DI_age', 'DI_race']:
            # df_full_result_selection[col] = df_full_result_selection[col].apply(lambda x: abs(1 - x))
           
           ### Calcul de valeurs moyenne pour le full 
           result_sel_avg_avg = df_full_result_selection_avg.mean()
           cost_sel_avg_avg = df_full_cost_selection_avg.mean()
           cost_time_sel_avg_avg = df_full_cost_time_selection_avg.mean()

           result_sel_std_avg = df_full_result_selection_std.mean()
           cost_sel_std_avg = df_full_cost_selection_std.mean()
           cost_time_sel_std_avg = df_full_cost_time_selection_std.mean()

           result_sel_vc_avg = df_full_result_selection_vc.mean()
           cost_sel_vc_avg = df_full_cost_selection_vc.mean()
           cost_time_sel_vc_avg = df_full_cost_time_selection_vc.mean()

           ## Faire les t-test
           # Cost
           # time
           test_time = evaluate_score(df_full_cost_time_selection_vc['Full_training_time'].to_numpy(), df_full_cost_time_vc['Full_training_time'].to_numpy()) 
           
           # Model quality
           # accuracy
           test_acc = evaluate_score(df_full_cost_selection_vc['accuracy'].to_numpy(), df_full_cost_vc['accuracy'].to_numpy()) 
           # F1-score
           test_f1 = evaluate_score(df_full_result_selection_vc['F1_score'].to_numpy(), df_full_result_vc['F1_score'].to_numpy()) 
           #precision
           test_precision = evaluate_score(df_full_result_selection_vc['Precision'].to_numpy(), df_full_result_vc['Precision'].to_numpy()) 
           # recall 
           test_recall = evaluate_score(df_full_result_selection_vc['Recall'].to_numpy(), df_full_result_vc['Recall'].to_numpy()) 

           # Fairness
           # SPD gender
           test_spd_g = evaluate_score(df_full_result_selection_vc['SPD_gender'].to_numpy(), df_full_result_vc['SPD_gender'].to_numpy())
           # EOD gender
           test_eod_g = evaluate_score(df_full_result_selection_vc['EOD_gender'].to_numpy(), df_full_result_vc['EOD_gender'].to_numpy())
           # AOD gender
           test_aod_g = evaluate_score(df_full_result_selection_vc['AOD_gender'].to_numpy(), df_full_result_vc['AOD_gender'].to_numpy())
           # DI gender
           test_di_g = evaluate_score(df_full_result_selection_vc['DI_gender'].to_numpy(), df_full_result_vc['DI_gender'].to_numpy())
           # DcI gender
           test_dci_g = evaluate_score(df_full_result_selection_vc['DcI_gender'].to_numpy(), df_full_result_vc['DcI_gender'].to_numpy())

           # SPD age
           test_spd_a = evaluate_score(df_full_result_selection_vc['SPD_age'].to_numpy(), df_full_result_vc['SPD_age'].to_numpy())
           # EOD gender
           test_eod_a = evaluate_score(df_full_result_selection_vc['EOD_age'].to_numpy(), df_full_result_vc['EOD_age'].to_numpy())
           # AOD gender
           test_aod_a = evaluate_score(df_full_result_selection_vc['AOD_age'].to_numpy(), df_full_result_vc['AOD_age'].to_numpy())
           # DI gender
           test_di_a = evaluate_score(df_full_result_selection_vc['DI_age'].to_numpy(), df_full_result_vc['DI_age'].to_numpy())
           # DcI gender
           test_dci_a = evaluate_score(df_full_result_selection_vc['DcI_age'].to_numpy(), df_full_result_vc['DcI_age'].to_numpy())

           # SPD race
           test_spd_r = evaluate_score(df_full_result_selection_vc['SPD_race'].to_numpy(), df_full_result_vc['SPD_race'].to_numpy())
           # EOD race
           test_eod_r = evaluate_score(df_full_result_selection_vc['EOD_race'].to_numpy(), df_full_result_vc['EOD_race'].to_numpy())
           # AOD race
           test_aod_r = evaluate_score(df_full_result_selection_vc['AOD_race'].to_numpy(), df_full_result_vc['AOD_race'].to_numpy())
           # DI race
           test_di_r = evaluate_score(df_full_result_selection_vc['DI_race'].to_numpy(), df_full_result_vc['DI_race'].to_numpy())
           # DcI race
           test_dci_r = evaluate_score(df_full_result_selection_vc['DcI_race'].to_numpy(), df_full_result_vc['DcI_race'].to_numpy())



           avg_sel_data = pd.DataFrame({
            **{str(col) + '_avg_avg': [result_sel_avg_avg[col]] for col in result_sel_avg_avg.index},
            **{str(col) + '_avg_avg': [cost_sel_avg_avg[col]] for col in cost_sel_avg_avg.index},
            **{str(col) + '_avg_avg': [cost_time_sel_avg_avg[col]] for col in cost_time_sel_avg_avg.index},
            **{str(col) + '_std_avg': [result_sel_std_avg[col]] for col in result_sel_std_avg.index},
            **{str(col) + '_std_avg': [cost_sel_std_avg[col]] for col in cost_sel_std_avg.index},
            **{str(col) + '_std_avg': [cost_time_sel_std_avg[col]] for col in cost_time_sel_std_avg.index},
            **{str(col) + '_vc_avg': [result_sel_vc_avg[col]] for col in result_sel_vc_avg.index},
            **{str(col) + '_vc_avg': [cost_sel_vc_avg[col]] for col in cost_sel_vc_avg.index},
            **{str(col) + '_vc_avg': [cost_time_sel_vc_avg[col]] for col in cost_time_sel_vc_avg.index},
           'dataset': ['adult'],
           'model': [model],
           'system': [system],
           'ratio': [directory_path_2.split('_')[-1]],
           'test_time' : [test_time], 
           'test_acc' : [test_acc], 
           'test_f1' : [test_f1], 
           'test_precision' : [test_precision], 
           'test_recall' : [test_recall],
           'test_SPD_gender' : [test_spd_g], 
           'test_EOD_gender' : [test_eod_g], 
           'test_AOD_gender' : [test_aod_g], 
           'test_DI_gender' : [test_di_g], 
           'test_DcI_gender' : [test_dci_g], 

           'test_SPD_age' : [test_spd_a],
           'test_EOD_age' : [test_eod_a],
           'test_AOD_age' : [test_aod_a], 
           'test_DI_age' : [test_di_a], 
           'test_DcI_age' : [test_dci_a], 

           'test_SPD_race' : [test_spd_r],
           'test_EOD_race' : [test_eod_r],
           'test_AOD_race' : [test_aod_r], 
           'test_DI_race' : [test_di_r],
           'test_DcI_race' : [test_dci_r],
            })
           
           df_average_results = pd.concat([df_average_results, avg_sel_data], ignore_index=True)




df_average_results.to_csv('../results/test/adult_ttest_results_epochs_intervals_vc_metrics.csv', index=False)


KDD dataset

In [ ]:
import os
import pandas as pd
import numpy as np
# Define paths and initialize variables
systems_path = ['../results/kdd-selection/CRAIGPB', '../results/kdd-selection/GLISTERPB', '../results/kdd-selection/GradMatchPB', '../results/kdd-selection/Random']
ratio_path = ['/kdd_0.05', '/kdd_0.1', '/kdd_0.2','/kdd_0.3', '/kdd_0.5']
models = ['Logreg', 'MLP', 'SVM']  

directory_full = '../results/kdd-selection/Full/kdd_1'


# Initialize full result and cost DataFrames
df_full_result_avg = pd.DataFrame()
df_full_cost_avg = pd.DataFrame()
df_full_cost_time_avg = pd.DataFrame()

df_full_result_std = pd.DataFrame()
df_full_cost_std = pd.DataFrame()
df_full_cost_time_std = pd.DataFrame()

df_full_result_vc = pd.DataFrame()
df_full_cost_vc = pd.DataFrame()
df_full_cost_time_vc = pd.DataFrame()

excluded_columns = ['DI_gender', 'DI_age', 'DI_race']

i = 0
files = []

df_average_results = pd.DataFrame()

# Load full system results for comparisons
for model in models: 

    df_full_result_avg = pd.DataFrame()
    df_full_cost_avg = pd.DataFrame()
    df_full_cost_time_avg = pd.DataFrame()

    df_full_result_std = pd.DataFrame()
    df_full_cost_std = pd.DataFrame()
    df_full_cost_time_std = pd.DataFrame()

    df_full_result_vc = pd.DataFrame()
    df_full_cost_vc = pd.DataFrame()
    df_full_cost_time_vc = pd.DataFrame()

    for filename in os.listdir(directory_full):
        if "fair_metrics_"+ model in filename :
           file_path = os.path.join(directory_full, filename)
           if model == 'MLP':
              df_r = pd.read_csv(file_path).iloc[34:79, 1:]
           else:
              df_r = pd.read_csv(file_path).iloc[24:79, 1:]
           df_r.update(df_r.drop(columns=excluded_columns, errors='ignore') * 100)
           df_r_vc = df_r.abs().std().to_frame().T / df_r.abs().mean()

           df_full_result_avg = pd.concat([df_full_result_avg, df_r.abs().mean().to_frame().T], ignore_index=True)
           df_full_result_std = pd.concat([df_full_result_std, df_r.abs().std().to_frame().T], ignore_index=True)
           df_full_result_vc = pd.concat([df_full_result_vc, df_r_vc], ignore_index=True)

        if "cost_metrics_" + model in filename and filename.endswith("_0.csv"):
            file_path = os.path.join(directory_full, filename)
            if model == 'MLP':
               df_ct = pd.read_csv(file_path).iloc[34:79, 1:]
            else:
               df_ct = pd.read_csv(file_path).iloc[24:79, 1:]

            df_ct_vc = df_ct.abs().std().to_frame().T / df_ct.abs().mean()

            df_full_cost_time_avg = pd.concat([df_full_cost_time_avg, df_ct.abs().mean().to_frame().T], ignore_index=True)
            df_full_cost_time_std = pd.concat([df_full_cost_time_std, df_ct.abs().std().to_frame().T], ignore_index=True)
            df_full_cost_time_vc = pd.concat([df_full_cost_time_vc, df_ct_vc], ignore_index=True)

        if "cost_metrics_" + model in filename :
            file_path = os.path.join(directory_full, filename)
            if model == 'MLP':
               df_c = pd.read_csv(file_path).iloc[34:79, 1:]
            else:
               df_c = pd.read_csv(file_path).iloc[24:79, 1:]

            df_c_vc = df_c.abs().std().to_frame().T / df_c.abs().mean()

            df_full_cost_avg = pd.concat([df_full_cost_avg, df_c.abs().mean().to_frame().T], ignore_index=True)
            df_full_cost_std = pd.concat([df_full_cost_std, df_c.abs().std().to_frame().T], ignore_index=True)
            df_full_cost_vc = pd.concat([df_full_cost_vc, df_c_vc], ignore_index=True)

    print('**********************')
    print(model)

    #for col in ['DI_gender', 'DI_age', 'DI_race']:
    #  df_full_result[col] = df_full_result[col].apply(lambda x: abs(1 - x))
    

    ### Calcul de valeurs moyenne pour le full 
    result_avg_avg = df_full_result_avg.mean()
    cost_avg_avg = df_full_cost_avg.mean()
    cost_time_avg_avg = df_full_cost_time_avg.mean()

    result_std_avg = df_full_result_std.mean()
    cost_std_avg = df_full_cost_std.mean()
    cost_time_std_avg = df_full_cost_time_std.mean()

    result_vc_avg = df_full_result_vc.mean()
    cost_vc_avg = df_full_cost_vc.mean()
    cost_time_vc_avg = df_full_cost_time_vc.mean()

    # Ligne de valeurs moyennes de full 
    avg_data = pd.DataFrame({
        **{str(col) + '_avg_avg': [result_avg_avg[col]] for col in result_avg_avg.index},
        **{str(col) + '_avg_avg': [cost_avg_avg[col]] for col in cost_avg_avg.index},
        **{str(col) + '_avg_avg': [cost_time_avg_avg[col]] for col in cost_time_avg_avg.index},
        **{str(col) + '_std_avg': [result_std_avg[col]] for col in result_std_avg.index},
        **{str(col) + '_std_avg': [cost_std_avg[col]] for col in cost_std_avg.index},
        **{str(col) + '_std_avg': [cost_time_std_avg[col]] for col in cost_time_std_avg.index},
        **{str(col) + '_vc_avg': [result_vc_avg[col]] for col in result_vc_avg.index},
        **{str(col) + '_vc_avg': [cost_vc_avg[col]] for col in cost_vc_avg.index},
        **{str(col) + '_vc_avg': [cost_time_vc_avg[col]] for col in cost_time_vc_avg.index},
        'dataset': ['kdd'],
        'model': [model],
        'system': ['Full'],
        'ratio': [1],
    })

    df_average_results = pd.concat([df_average_results, avg_data], ignore_index=True)

    # Lire les fichiers de système de selection par système et par ratio

    for directory_path_1 in systems_path:
       for directory_path_2 in ratio_path:
           directory = directory_path_1 + directory_path_2
           system = directory_path_1.split('/')[-1]           
        
           df_full_result_selection_avg = pd.DataFrame()
           df_full_cost_selection_avg = pd.DataFrame()
           df_full_cost_time_selection_avg = pd.DataFrame()

           df_full_result_selection_std = pd.DataFrame()
           df_full_cost_selection_std = pd.DataFrame()
           df_full_cost_time_selection_std = pd.DataFrame()

           df_full_result_selection_vc = pd.DataFrame()
           df_full_cost_selection_vc = pd.DataFrame()
           df_full_cost_time_selection_vc = pd.DataFrame()
        
           # Cost file processing
           for filename in os.listdir(directory):
                if "fair_metrics_" + model in filename:
                    file_path = os.path.join(directory, filename)
                    if model == 'MLP':
                       df_to_add_f = pd.read_csv(file_path).iloc[14:59, 1:].abs()
                    else:
                       df_to_add_f = pd.read_csv(file_path).iloc[4:59, 1:].abs()
                    df_to_add_f.update(df_to_add_f.drop(columns=excluded_columns, errors='ignore') * 100)

                    df_to_add_f_vc = df_to_add_f.abs().std().to_frame().T / df_to_add_f.abs().mean()

                    df_full_result_selection_avg = pd.concat([df_full_result_selection_avg, df_to_add_f.mean().to_frame().T], ignore_index=True)
                    df_full_result_selection_std = pd.concat([df_full_result_selection_std, df_to_add_f.std().to_frame().T], ignore_index=True)
                    df_full_result_selection_vc = pd.concat([df_full_result_selection_vc, df_to_add_f_vc], ignore_index=True)
                    

                if (("cost_metrics_" + model in filename) and (filename.endswith("_0.csv"))):
                    file_path = os.path.join(directory, filename)
                    if model == 'MLP':
                       df_to_add_t = pd.read_csv(file_path).iloc[14:59, 1:].abs()
                    else:
                       df_to_add_t = pd.read_csv(file_path).iloc[4:59, 1:].abs()

                    df_to_add_t_vc = df_to_add_t.abs().std().to_frame().T / df_to_add_t.abs().mean()

                    df_full_cost_time_selection_avg = pd.concat([df_full_cost_time_selection_avg, df_to_add_t.mean().to_frame().T], ignore_index=True)
                    df_full_cost_time_selection_std = pd.concat([df_full_cost_time_selection_std, df_to_add_t.std().to_frame().T], ignore_index=True)
                    df_full_cost_time_selection_vc = pd.concat([df_full_cost_time_selection_vc, df_to_add_t_vc], ignore_index=True)
    

        
                if "cost_metrics_" + model in filename:
                    file_path = os.path.join(directory, filename)
                    if model == 'MLP':
                       df_to_add_c = pd.read_csv(file_path).iloc[14:59, 1:].abs()
                    else:
                       df_to_add_c = pd.read_csv(file_path).iloc[4:59, 1:].abs()

                    df_to_add_c_vc = df_to_add_c.abs().std().to_frame().T / df_to_add_c.abs().mean()

                    df_full_cost_selection_avg = pd.concat([df_full_cost_selection_avg, df_to_add_c.mean().to_frame().T], ignore_index=True)
                    df_full_cost_selection_std = pd.concat([df_full_cost_selection_std, df_to_add_c.std().to_frame().T], ignore_index=True)
                    df_full_cost_selection_vc = pd.concat([df_full_cost_selection_vc, df_to_add_c_vc], ignore_index=True)
           
                
           #for col in ['DI_gender', 'DI_age', 'DI_race']:
           #  df_full_result_selection[col] = df_full_result_selection[col].apply(lambda x: abs(1 - x))

           ### Calcul de valeurs moyenne pour le full 
           result_sel_avg_avg = df_full_result_selection_avg.mean()
           cost_sel_avg_avg = df_full_cost_selection_avg.mean()
           cost_time_sel_avg_avg = df_full_cost_time_selection_avg.mean()

           result_sel_std_avg = df_full_result_selection_std.mean()
           cost_sel_std_avg = df_full_cost_selection_std.mean()
           cost_time_sel_std_avg = df_full_cost_time_selection_std.mean()

           result_sel_vc_avg = df_full_result_selection_vc.mean()
           cost_sel_vc_avg = df_full_cost_selection_vc.mean()
           cost_time_sel_vc_avg = df_full_cost_time_selection_vc.mean()

           ## Faire les t-test
           # Cost
           # time
           test_time = evaluate_score(df_full_cost_time_selection_vc['Full_training_time'].to_numpy(), df_full_cost_time_vc['Full_training_time'].to_numpy()) 
           
           # Model quality
           # accuracy
           test_acc = evaluate_score(df_full_cost_selection_vc['accuracy'].to_numpy(), df_full_cost_vc['accuracy'].to_numpy()) 
           # F1-score
           test_f1 = evaluate_score(df_full_result_selection_vc['F1_score'].to_numpy(), df_full_result_vc['F1_score'].to_numpy()) 
           #precision
           test_precision = evaluate_score(df_full_result_selection_vc['Precision'].to_numpy(), df_full_result_vc['Precision'].to_numpy()) 
           # recall 
           test_recall = evaluate_score(df_full_result_selection_vc['Recall'].to_numpy(), df_full_result_vc['Recall'].to_numpy()) 

           # Fairness
           # SPD gender
           test_spd_g = evaluate_score(df_full_result_selection_vc['SPD_gender'].to_numpy(), df_full_result_vc['SPD_gender'].to_numpy())
           # EOD gender
           test_eod_g = evaluate_score(df_full_result_selection_vc['EOD_gender'].to_numpy(), df_full_result_vc['EOD_gender'].to_numpy())
           # AOD gender
           test_aod_g = evaluate_score(df_full_result_selection_vc['AOD_gender'].to_numpy(), df_full_result_vc['AOD_gender'].to_numpy())
           # DI gender
           test_di_g = evaluate_score(df_full_result_selection_vc['DI_gender'].to_numpy(), df_full_result_vc['DI_gender'].to_numpy())
           # DcI gender
           test_dci_g = evaluate_score(df_full_result_selection_vc['DcI_gender'].to_numpy(), df_full_result_vc['DcI_gender'].to_numpy())

           # SPD age
           test_spd_a = evaluate_score(df_full_result_selection_vc['SPD_age'].to_numpy(), df_full_result_vc['SPD_age'].to_numpy())
           # EOD gender
           test_eod_a = evaluate_score(df_full_result_selection_vc['EOD_age'].to_numpy(), df_full_result_vc['EOD_age'].to_numpy())
           # AOD gender
           test_aod_a = evaluate_score(df_full_result_selection_vc['AOD_age'].to_numpy(), df_full_result_vc['AOD_age'].to_numpy())
           # DI gender
           test_di_a = evaluate_score(df_full_result_selection_vc['DI_age'].to_numpy(), df_full_result_vc['DI_age'].to_numpy())
           # DcI gender
           test_dci_a = evaluate_score(df_full_result_selection_vc['DcI_age'].to_numpy(), df_full_result_vc['DcI_age'].to_numpy())

           # SPD race
           test_spd_r = evaluate_score(df_full_result_selection_vc['SPD_race'].to_numpy(), df_full_result_vc['SPD_race'].to_numpy())
           # EOD race
           test_eod_r = evaluate_score(df_full_result_selection_vc['EOD_race'].to_numpy(), df_full_result_vc['EOD_race'].to_numpy())
           # AOD race
           test_aod_r = evaluate_score(df_full_result_selection_vc['AOD_race'].to_numpy(), df_full_result_vc['AOD_race'].to_numpy())
           # DI race
           test_di_r = evaluate_score(df_full_result_selection_vc['DI_race'].to_numpy(), df_full_result_vc['DI_race'].to_numpy())
           # DcI race
           test_dci_r = evaluate_score(df_full_result_selection_vc['DcI_race'].to_numpy(), df_full_result_vc['DcI_race'].to_numpy())



           avg_sel_data = pd.DataFrame({
            **{str(col) + '_avg_avg': [result_sel_avg_avg[col]] for col in result_sel_avg_avg.index},
            **{str(col) + '_avg_avg': [cost_sel_avg_avg[col]] for col in cost_sel_avg_avg.index},
            **{str(col) + '_avg_avg': [cost_time_sel_avg_avg[col]] for col in cost_time_sel_avg_avg.index},
            **{str(col) + '_std_avg': [result_sel_std_avg[col]] for col in result_sel_std_avg.index},
            **{str(col) + '_std_avg': [cost_sel_std_avg[col]] for col in cost_sel_std_avg.index},
            **{str(col) + '_std_avg': [cost_time_sel_std_avg[col]] for col in cost_time_sel_std_avg.index},
            **{str(col) + '_vc_avg': [result_sel_vc_avg[col]] for col in result_sel_vc_avg.index},
            **{str(col) + '_vc_avg': [cost_sel_vc_avg[col]] for col in cost_sel_vc_avg.index},
            **{str(col) + '_vc_avg': [cost_time_sel_vc_avg[col]] for col in cost_time_sel_vc_avg.index},
           'dataset': ['kdd'],
           'model': [model],
           'system': [system],
           'ratio': [directory_path_2.split('_')[-1]],
           'test_time' : [test_time], 
           'test_acc' : [test_acc], 
           'test_f1' : [test_f1], 
           'test_precision' : [test_precision], 
           'test_recall' : [test_recall],
           'test_SPD_gender' : [test_spd_g], 
           'test_EOD_gender' : [test_eod_g], 
           'test_AOD_gender' : [test_aod_g], 
           'test_DI_gender' : [test_di_g], 
           'test_DcI_gender' : [test_dci_g], 

           'test_SPD_age' : [test_spd_a],
           'test_EOD_age' : [test_eod_a],
           'test_AOD_age' : [test_aod_a], 
           'test_DI_age' : [test_di_a], 
           'test_DcI_age' : [test_dci_a], 

           'test_SPD_race' : [test_spd_r],
           'test_EOD_race' : [test_eod_r],
           'test_AOD_race' : [test_aod_r], 
           'test_DI_race' : [test_di_r],
           'test_DcI_race' : [test_dci_r],
            })
           
           df_average_results = pd.concat([df_average_results, avg_sel_data], ignore_index=True)




 

df_average_results.to_csv('../results/test/kdd_ttest_results_epochs_intervals_vc_metrics.csv', index=False)



DC dataset

In [ ]:
import os
import pandas as pd
import numpy as np
# Define paths and initialize variables
systems_path = ['../results/dc-selection/CRAIGPB', '../results/dc-selection/GLISTERPB', '../results/dc-selection/GradMatchPB', '../results/dc-selection/Random']
ratio_path = ['/dc_0.05', '/dc_0.1', '/dc_0.2','/dc_0.3', '/dc_0.5']
models = ['LogReg', 'MLP', 'SVM']  

directory_full = '../results/dc-selection/Full/dc_1'


# Initialize full result and cost DataFrames
df_full_result_avg = pd.DataFrame()
df_full_cost_avg = pd.DataFrame()
df_full_cost_time_avg = pd.DataFrame()

df_full_result_std = pd.DataFrame()
df_full_cost_std = pd.DataFrame()
df_full_cost_time_std = pd.DataFrame()

df_full_result_vc = pd.DataFrame()
df_full_cost_vc = pd.DataFrame()
df_full_cost_time_vc = pd.DataFrame()

excluded_columns = ['DI_gender', 'DI_age', 'DI_race']

i = 0
files = []

df_average_results = pd.DataFrame()

# Load full system results for comparisons
for model in models: 

    df_full_result_avg = pd.DataFrame()
    df_full_cost_avg = pd.DataFrame()
    df_full_cost_time_avg = pd.DataFrame()

    df_full_result_std = pd.DataFrame()
    df_full_cost_std = pd.DataFrame()
    df_full_cost_time_std = pd.DataFrame()

    df_full_result_vc = pd.DataFrame()
    df_full_cost_vc = pd.DataFrame()
    df_full_cost_time_vc = pd.DataFrame()

    for filename in os.listdir(directory_full):
        if "fair_metrics_"+ model in filename :
           file_path = os.path.join(directory_full, filename)
           if model == 'MLP':
               df_r = pd.read_csv(file_path).iloc[44:99, 1:]
           elif model == 'SVM':
               df_r = pd.read_csv(file_path).iloc[29:99, 1:]
           else:
               df_r = pd.read_csv(file_path).iloc[124:269, 1:]
           df_r.update(df_r.drop(columns=excluded_columns, errors='ignore') * 100)

           df_r_vc = df_r.abs().std().to_frame().T / df_r.abs().mean()

           df_full_result_avg = pd.concat([df_full_result_avg, df_r.abs().mean().to_frame().T], ignore_index=True)
           df_full_result_std = pd.concat([df_full_result_std, df_r.abs().std().to_frame().T], ignore_index=True)
           df_full_result_vc = pd.concat([df_full_result_vc, df_r_vc], ignore_index=True)

        if "cost_metrics_" + model in filename and filename.endswith("_0.csv"):
            file_path = os.path.join(directory_full, filename)
            if model == 'MLP':
               df_ct = pd.read_csv(file_path).iloc[44:99, 1:]
            elif model == 'SVM':
               df_ct = pd.read_csv(file_path).iloc[29:99, 1:]
            else:
               df_ct = pd.read_csv(file_path).iloc[124:269, 1:]

            df_ct_vc = df_ct.abs().std().to_frame().T / df_ct.abs().mean()

            df_full_cost_time_avg = pd.concat([df_full_cost_time_avg, df_ct.abs().mean().to_frame().T], ignore_index=True)
            df_full_cost_time_std = pd.concat([df_full_cost_time_std, df_ct.abs().std().to_frame().T], ignore_index=True)
            df_full_cost_time_vc = pd.concat([df_full_cost_time_vc, df_ct_vc], ignore_index=True)

        if "cost_metrics_" + model in filename :
            file_path = os.path.join(directory_full, filename)
            if model == 'MLP':
               df_c = pd.read_csv(file_path).iloc[44:99, 1:]
            elif model == 'SVM':
               df_c = pd.read_csv(file_path).iloc[29:99, 1:]
            else:
               df_c = pd.read_csv(file_path).iloc[124:269, 1:]

            df_c_vc = df_c.abs().std().to_frame().T / df_c.abs().mean()

            df_full_cost_avg = pd.concat([df_full_cost_avg, df_c.abs().mean().to_frame().T], ignore_index=True)
            df_full_cost_std = pd.concat([df_full_cost_std, df_c.abs().std().to_frame().T], ignore_index=True)
            df_full_cost_vc = pd.concat([df_full_cost_vc, df_c_vc], ignore_index=True)

    print('**********************')
    print(model)

    #for col in ['DI_gender', 'DI_age']:
    #  df_full_result[col] = df_full_result[col].apply(lambda x: abs(1 - x))
    

    ### Calcul de valeurs moyenne pour le full 
    result_avg_avg = df_full_result_avg.mean()
    cost_avg_avg = df_full_cost_avg.mean()
    cost_time_avg_avg = df_full_cost_time_avg.mean()

    result_std_avg = df_full_result_std.mean()
    cost_std_avg = df_full_cost_std.mean()
    cost_time_std_avg = df_full_cost_time_std.mean()

    result_vc_avg = df_full_result_vc.mean()
    cost_vc_avg = df_full_cost_vc.mean()
    cost_time_vc_avg = df_full_cost_time_vc.mean()

    # Ligne de valeurs moyennes de full 
    avg_data = pd.DataFrame({
        **{str(col) + '_avg_avg': [result_avg_avg[col]] for col in result_avg_avg.index},
        **{str(col) + '_avg_avg': [cost_avg_avg[col]] for col in cost_avg_avg.index},
        **{str(col) + '_avg_avg': [cost_time_avg_avg[col]] for col in cost_time_avg_avg.index},
        **{str(col) + '_std_avg': [result_std_avg[col]] for col in result_std_avg.index},
        **{str(col) + '_std_avg': [cost_std_avg[col]] for col in cost_std_avg.index},
        **{str(col) + '_std_avg': [cost_time_std_avg[col]] for col in cost_time_std_avg.index},
        **{str(col) + '_vc_avg': [result_vc_avg[col]] for col in result_vc_avg.index},
        **{str(col) + '_vc_avg': [cost_vc_avg[col]] for col in cost_vc_avg.index},
        **{str(col) + '_vc_avg': [cost_time_vc_avg[col]] for col in cost_time_vc_avg.index},
        'dataset': ['dc'],
        'model': [model],
        'system': ['Full'],
        'ratio': [1],
    })

    df_average_results = pd.concat([df_average_results, avg_data], ignore_index=True)

    # Lire les fichiers de système de selection par système et par ratio

    for directory_path_1 in systems_path:
       for directory_path_2 in ratio_path:
           directory = directory_path_1 + directory_path_2
           system = directory_path_1.split('/')[-1]           
        
           df_full_result_selection_avg = pd.DataFrame()
           df_full_cost_selection_avg = pd.DataFrame()
           df_full_cost_time_selection_avg = pd.DataFrame()

           df_full_result_selection_std = pd.DataFrame()
           df_full_cost_selection_std = pd.DataFrame()
           df_full_cost_time_selection_std = pd.DataFrame()

           df_full_result_selection_vc = pd.DataFrame()
           df_full_cost_selection_vc = pd.DataFrame()
           df_full_cost_time_selection_vc = pd.DataFrame()
        
           
           # Cost file processing
           for filename in os.listdir(directory):
                if "fair_metrics_" + model in filename:
                    file_path = os.path.join(directory, filename)
                    if model == 'MLP': 
                        df_to_add_f = pd.read_csv(file_path).iloc[44:99, 1:].abs()
                    elif model == 'SVM':
                        df_to_add_f = pd.read_csv(file_path).iloc[29:99, 1:].abs()
                    else:
                        df_to_add_f = pd.read_csv(file_path).iloc[124:269, 1:].abs()
                    df_to_add_f.update(df_to_add_f.drop(columns=excluded_columns, errors='ignore') * 100)

                    df_to_add_f_vc = df_to_add_f.abs().std().to_frame().T / df_to_add_f.abs().mean()

                    df_full_result_selection_avg = pd.concat([df_full_result_selection_avg, df_to_add_f.mean().to_frame().T], ignore_index=True)
                    df_full_result_selection_std = pd.concat([df_full_result_selection_std, df_to_add_f.std().to_frame().T], ignore_index=True)
                    df_full_result_selection_vc = pd.concat([df_full_result_selection_vc, df_to_add_f_vc], ignore_index=True)
                    

                if (("cost_metrics_" + model in filename) and (filename.endswith("_0.csv"))):
                    file_path = os.path.join(directory, filename)
                    if model == 'MLP': 
                        df_to_add_t = pd.read_csv(file_path).iloc[44:99, 1:].abs()
                    elif model == 'SVM':
                        df_to_add_t = pd.read_csv(file_path).iloc[29:99, 1:].abs()
                    else: 
                        df_to_add_t = pd.read_csv(file_path).iloc[124:269, 1:].abs()

                    df_to_add_t_vc = df_to_add_t.abs().std().to_frame().T / df_to_add_t.abs().mean()

                    df_full_cost_time_selection_avg = pd.concat([df_full_cost_time_selection_avg, df_to_add_t.mean().to_frame().T], ignore_index=True)
                    df_full_cost_time_selection_std = pd.concat([df_full_cost_time_selection_std, df_to_add_t.std().to_frame().T], ignore_index=True)
                    df_full_cost_time_selection_vc = pd.concat([df_full_cost_time_selection_vc, df_to_add_t_vc], ignore_index=True)
    

                

                if "cost_metrics_" + model in filename:
                    file_path = os.path.join(directory, filename)
                    if model == 'MLP': 
                        df_to_add_c = pd.read_csv(file_path).iloc[44:99, 1:].abs()
                    elif model == 'SVM':
                        df_to_add_c = pd.read_csv(file_path).iloc[29:99, 1:].abs()
                    else:
                        df_to_add_c = pd.read_csv(file_path).iloc[124:269, 1:].abs()
                    
                    df_to_add_c_vc = df_to_add_c.abs().std().to_frame().T / df_to_add_c.abs().mean()

                    df_full_cost_selection_avg = pd.concat([df_full_cost_selection_avg, df_to_add_c.mean().to_frame().T], ignore_index=True)
                    df_full_cost_selection_std = pd.concat([df_full_cost_selection_std, df_to_add_c.std().to_frame().T], ignore_index=True)
                    df_full_cost_selection_vc = pd.concat([df_full_cost_selection_vc, df_to_add_c_vc], ignore_index=True)
           
                
           #for col in ['DI_gender', 'DI_age']:
            # df_full_result_selection[col] = df_full_result_selection[col].apply(lambda x: abs(1 - x))
           
           ### Calcul de valeurs moyenne pour le full
           result_sel_avg_avg = df_full_result_selection_avg.mean()
           cost_sel_avg_avg = df_full_cost_selection_avg.mean()
           cost_time_sel_avg_avg = df_full_cost_time_selection_avg.mean()

           result_sel_std_avg = df_full_result_selection_std.mean()
           cost_sel_std_avg = df_full_cost_selection_std.mean()
           cost_time_sel_std_avg = df_full_cost_time_selection_std.mean()

           result_sel_vc_avg = df_full_result_selection_vc.mean()
           cost_sel_vc_avg = df_full_cost_selection_vc.mean()
           cost_time_sel_vc_avg = df_full_cost_time_selection_vc.mean()

           ## Faire les t-test
           # Cost
           # time
           test_time = evaluate_score(df_full_cost_time_selection_vc['Full_training_time'].to_numpy(), df_full_cost_time_vc['Full_training_time'].to_numpy()) 
           
           # Model quality
           # accuracy
           test_acc = evaluate_score(df_full_cost_selection_vc['accuracy'].to_numpy(), df_full_cost_vc['accuracy'].to_numpy()) 
           # F1-score
           test_f1 = evaluate_score(df_full_result_selection_vc['F1_score'].to_numpy(), df_full_result_vc['F1_score'].to_numpy()) 
           #precision
           test_precision = evaluate_score(df_full_result_selection_vc['Precision'].to_numpy(), df_full_result_vc['Precision'].to_numpy()) 
           # recall 
           test_recall = evaluate_score(df_full_result_selection_vc['Recall'].to_numpy(), df_full_result_vc['Recall'].to_numpy()) 

           # Fairness
           # SPD gender
           test_spd_g = evaluate_score(df_full_result_selection_vc['SPD_gender'].to_numpy(), df_full_result_vc['SPD_gender'].to_numpy())
           # EOD gender
           test_eod_g = evaluate_score(df_full_result_selection_vc['EOD_gender'].to_numpy(), df_full_result_vc['EOD_gender'].to_numpy())
           # AOD gender
           test_aod_g = evaluate_score(df_full_result_selection_vc['AOD_gender'].to_numpy(), df_full_result_vc['AOD_gender'].to_numpy())
           # DI gender
           test_di_g = evaluate_score(df_full_result_selection_vc['DI_gender'].to_numpy(), df_full_result_vc['DI_gender'].to_numpy())
           # DcI gender
           test_dci_g = evaluate_score(df_full_result_selection_vc['DcI_gender'].to_numpy(), df_full_result_vc['DcI_gender'].to_numpy())

           # SPD age
           test_spd_a = evaluate_score(df_full_result_selection_vc['SPD_age'].to_numpy(), df_full_result_vc['SPD_age'].to_numpy())
           # EOD age
           test_eod_a = evaluate_score(df_full_result_selection_vc['EOD_age'].to_numpy(), df_full_result_vc['EOD_age'].to_numpy())
           # AOD age
           test_aod_a = evaluate_score(df_full_result_selection_vc['AOD_age'].to_numpy(), df_full_result_vc['AOD_age'].to_numpy())
           # DI age
           test_di_a = evaluate_score(df_full_result_selection_vc['DI_age'].to_numpy(), df_full_result_vc['DI_age'].to_numpy())
           # DcI age
           test_dci_a = evaluate_score(df_full_result_selection_vc['DcI_age'].to_numpy(), df_full_result_vc['DcI_age'].to_numpy())

        



           avg_sel_data = pd.DataFrame({
            **{str(col) + '_avg_avg': [result_sel_avg_avg[col]] for col in result_sel_avg_avg.index},
            **{str(col) + '_avg_avg': [cost_sel_avg_avg[col]] for col in cost_sel_avg_avg.index},
            **{str(col) + '_avg_avg': [cost_time_sel_avg_avg[col]] for col in cost_time_sel_avg_avg.index},
            **{str(col) + '_std_avg': [result_sel_std_avg[col]] for col in result_sel_std_avg.index},
            **{str(col) + '_std_avg': [cost_sel_std_avg[col]] for col in cost_sel_std_avg.index},
            **{str(col) + '_std_avg': [cost_time_sel_std_avg[col]] for col in cost_time_sel_std_avg.index},
            **{str(col) + '_vc_avg': [result_sel_vc_avg[col]] for col in result_sel_vc_avg.index},
            **{str(col) + '_vc_avg': [cost_sel_vc_avg[col]] for col in cost_sel_vc_avg.index},
            **{str(col) + '_vc_avg': [cost_time_sel_vc_avg[col]] for col in cost_time_sel_vc_avg.index},
           'dataset': ['dc'],
           'model': [model],
           'system': [system],
           'ratio': [directory_path_2.split('_')[-1]],
           'test_time' : [test_time], 
           'test_acc' : [test_acc], 
           'test_f1' : [test_f1], 
           'test_precision' : [test_precision], 
           'test_recall' : [test_recall],
           'test_SPD_gender' : [test_spd_g], 
           'test_EOD_gender' : [test_eod_g], 
           'test_AOD_gender' : [test_aod_g], 
           'test_DI_gender' : [test_di_g], 
           'test_DcI_gender' : [test_dci_g], 

           'test_SPD_age' : [test_spd_a],
           'test_EOD_age' : [test_eod_a],
           'test_AOD_age' : [test_aod_a], 
           'test_DI_age' : [test_di_a], 
           'test_DcI_age' : [test_dci_a], 

           'test_SPD_race' : ' ',
           'test_EOD_race' : ' ',
           'test_AOD_race' : ' ', 
           'test_DI_race' : ' ',
           'test_DcI_race' : ' ',
            })
           
           df_average_results = pd.concat([df_average_results, avg_sel_data], ignore_index=True)


df_average_results.to_csv('../results/test/dc_ttest_results_epochs_intervals_vc_metrics.csv', index=False)



MobiAct dataset

In [ ]:
import os
import pandas as pd
import numpy as np
# Define paths and initialize variables
systems_path = ['../results/mobiact-selection/CRAIGPB', '../results/mobiact-selection/GLISTERPB', '../results/mobiact-selection/GradMatchPB', '../results/mobiact-selection/Random']
ratio_path = ['/mobiact_0.05', '/mobiact_0.1', '/mobiact_0.2','/mobiact_0.3', '/mobiact_0.5']
models = ['MLP']  

directory_full = '../results/mobiact-selection/Full/mobiact_1'


# Initialize full result and cost DataFrames
df_full_result_avg = pd.DataFrame()
df_full_cost_avg = pd.DataFrame()
df_full_cost_time_avg = pd.DataFrame()

df_full_result_std = pd.DataFrame()
df_full_cost_std = pd.DataFrame()
df_full_cost_time_std = pd.DataFrame()

df_full_result_vc = pd.DataFrame()
df_full_cost_vc = pd.DataFrame()
df_full_cost_time_vc = pd.DataFrame()

excluded_columns = ['DI_gender', 'DI_age', 'DI_race']

i = 0
files = []

df_average_results = pd.DataFrame()

# Load full system results for comparisons
for model in models: 

    df_full_result_avg = pd.DataFrame()
    df_full_cost_avg = pd.DataFrame()
    df_full_cost_time_avg = pd.DataFrame()

    df_full_result_std = pd.DataFrame()
    df_full_cost_std = pd.DataFrame()
    df_full_cost_time_std = pd.DataFrame()

    df_full_result_vc = pd.DataFrame()
    df_full_cost_vc = pd.DataFrame()
    df_full_cost_time_vc = pd.DataFrame()

    for filename in os.listdir(directory_full):
        if "train_mobiact_fair_metrics_" in filename :
           file_path = os.path.join(directory_full, filename)
           df_r = pd.read_csv(file_path).iloc[29:79, 1:]
           df_r.update(df_r.drop(columns=excluded_columns, errors='ignore') * 100)

           df_r_vc = df_r.abs().std().to_frame().T / df_r.abs().mean()

           df_full_result_avg = pd.concat([df_full_result_avg, df_r.abs().mean().to_frame().T], ignore_index=True)
           df_full_result_std = pd.concat([df_full_result_std, df_r.abs().std().to_frame().T], ignore_index=True)
           df_full_result_vc = pd.concat([df_full_result_vc, df_r_vc], ignore_index=True)

        if "train_mobiact_cost_metrics_"  in filename and filename.endswith("_0.csv"):
            file_path = os.path.join(directory_full, filename)
            df_ct = pd.read_csv(file_path).iloc[29:79, 1:]

            df_ct_vc = df_ct.abs().std().to_frame().T / df_ct.abs().mean()

            df_full_cost_time_avg = pd.concat([df_full_cost_time_avg, df_ct.abs().mean().to_frame().T], ignore_index=True)
            df_full_cost_time_std = pd.concat([df_full_cost_time_std, df_ct.abs().std().to_frame().T], ignore_index=True)
            df_full_cost_time_vc = pd.concat([df_full_cost_time_vc, df_ct_vc], ignore_index=True)

        if "train_mobiact_cost_metrics_"  in filename :
            file_path = os.path.join(directory_full, filename)
            df_c = pd.read_csv(file_path).iloc[29:79, 1:]

            df_c_vc = df_c.abs().std().to_frame().T / df_c.abs().mean()
            
            df_full_cost_avg = pd.concat([df_full_cost_avg, df_c.abs().mean().to_frame().T], ignore_index=True)
            df_full_cost_std = pd.concat([df_full_cost_std, df_c.abs().std().to_frame().T], ignore_index=True)
            df_full_cost_vc = pd.concat([df_full_cost_vc, df_c_vc], ignore_index=True)

    print('**********************')
    print(model)

    #for col in ['DI_gender', 'DI_age']:
    #  df_full_result[col] = df_full_result[col].apply(lambda x: abs(1 - x))

    ### Calcul de valeurs moyenne pour le full 
    result_avg_avg = df_full_result_avg.mean()
    cost_avg_avg = df_full_cost_avg.mean()
    cost_time_avg_avg = df_full_cost_time_avg.mean()

    result_std_avg = df_full_result_std.mean()
    cost_std_avg = df_full_cost_std.mean()
    cost_time_std_avg = df_full_cost_time_std.mean()

    result_vc_avg = df_full_result_vc.mean()
    cost_vc_avg = df_full_cost_vc.mean()
    cost_time_vc_avg = df_full_cost_time_vc.mean()

    # Ligne de valeurs moyennes de full 
    avg_data = pd.DataFrame({
        **{str(col) + '_avg_avg': [result_avg_avg[col]] for col in result_avg_avg.index},
        **{str(col) + '_avg_avg': [cost_avg_avg[col]] for col in cost_avg_avg.index},
        **{str(col) + '_avg_avg': [cost_time_avg_avg[col]] for col in cost_time_avg_avg.index},
        **{str(col) + '_std_avg': [result_std_avg[col]] for col in result_std_avg.index},
        **{str(col) + '_std_avg': [cost_std_avg[col]] for col in cost_std_avg.index},
        **{str(col) + '_std_avg': [cost_time_std_avg[col]] for col in cost_time_std_avg.index},
        **{str(col) + '_vc_avg': [result_vc_avg[col]] for col in result_vc_avg.index},
        **{str(col) + '_vc_avg': [cost_vc_avg[col]] for col in cost_vc_avg.index},
        **{str(col) + '_vc_avg': [cost_time_vc_avg[col]] for col in cost_time_vc_avg.index},
        'dataset': ['mobiact'],
        'model': [model],
        'system': ['Full'],
        'ratio': [1],
    })

    df_average_results = pd.concat([df_average_results, avg_data], ignore_index=True)

    # Lire les fichiers de système de selection par système et par ratio

    for directory_path_1 in systems_path:
       for directory_path_2 in ratio_path:
           directory = directory_path_1 + directory_path_2
           system = directory_path_1.split('/')[-1]           
        
           df_full_result_selection_avg = pd.DataFrame()
           df_full_cost_selection_avg = pd.DataFrame()
           df_full_cost_time_selection_avg = pd.DataFrame()

           df_full_result_selection_std = pd.DataFrame()
           df_full_cost_selection_std = pd.DataFrame()
           df_full_cost_time_selection_std = pd.DataFrame()

           df_full_result_selection_vc = pd.DataFrame()
           df_full_cost_selection_vc = pd.DataFrame()
           df_full_cost_time_selection_vc = pd.DataFrame()
        
           
           # Cost file processing
           for filename in os.listdir(directory):
                if "train_mobiact_fair_metrics_"  in filename:
                    file_path = os.path.join(directory, filename)
                    df_to_add_f = pd.read_csv(file_path).iloc[29:79, 1:].abs()
                    df_to_add_f.update(df_to_add_f.drop(columns=excluded_columns, errors='ignore') * 100)
                    
                    df_to_add_f_vc = df_to_add_f.abs().std().to_frame().T / df_to_add_f.abs().mean()

                    df_full_result_selection_avg = pd.concat([df_full_result_selection_avg, df_to_add_f.mean().to_frame().T], ignore_index=True)
                    df_full_result_selection_std = pd.concat([df_full_result_selection_std, df_to_add_f.std().to_frame().T], ignore_index=True)
                    df_full_result_selection_vc = pd.concat([df_full_result_selection_vc, df_to_add_f_vc], ignore_index=True)
                    

                if (("train_mobiact_cost_metrics_"  in filename) and (filename.endswith("_0.csv"))):
                    file_path = os.path.join(directory, filename)
                    df_to_add_t = pd.read_csv(file_path).iloc[29:79, 1:].abs()
                    
                    df_to_add_t_vc = df_to_add_t.abs().std().to_frame().T / df_to_add_t.abs().mean()

                    df_full_cost_time_selection_avg = pd.concat([df_full_cost_time_selection_avg, df_to_add_t.mean().to_frame().T], ignore_index=True)
                    df_full_cost_time_selection_std = pd.concat([df_full_cost_time_selection_std, df_to_add_t.std().to_frame().T], ignore_index=True)
                    df_full_cost_time_selection_vc = pd.concat([df_full_cost_time_selection_vc, df_to_add_t_vc], ignore_index=True)
    

                

                if "train_mobiact_cost_metrics_"  in filename:
                    file_path = os.path.join(directory, filename)
                    df_to_add_c = pd.read_csv(file_path).iloc[29:79, 1:].abs()

                    df_to_add_c_vc = df_to_add_c.abs().std().to_frame().T / df_to_add_c.abs().mean()
                    
                    df_full_cost_selection_avg = pd.concat([df_full_cost_selection_avg, df_to_add_c.mean().to_frame().T], ignore_index=True)
                    df_full_cost_selection_std = pd.concat([df_full_cost_selection_std, df_to_add_c.std().to_frame().T], ignore_index=True)
                    df_full_cost_selection_vc = pd.concat([df_full_cost_selection_vc, df_to_add_c_vc], ignore_index=True)
           
                
           #for col in ['DI_gender', 'DI_age']:
           #  df_full_result_selection[col] = df_full_result_selection[col].apply(lambda x: abs(1 - x))
    
           ### Calcul de valeurs moyenne pour le full
           result_sel_avg_avg = df_full_result_selection_avg.mean()
           cost_sel_avg_avg = df_full_cost_selection_avg.mean()
           cost_time_sel_avg_avg = df_full_cost_time_selection_avg.mean()

           result_sel_std_avg = df_full_result_selection_std.mean()
           cost_sel_std_avg = df_full_cost_selection_std.mean()
           cost_time_sel_std_avg = df_full_cost_time_selection_std.mean()

           result_sel_vc_avg = df_full_result_selection_vc.mean()
           cost_sel_vc_avg = df_full_cost_selection_vc.mean()
           cost_time_sel_vc_avg = df_full_cost_time_selection_vc.mean()

           ## Faire les t-test
           # Cost
           # time
           test_time = evaluate_score(df_full_cost_time_selection_vc['Full_training_time'].to_numpy(), df_full_cost_time_vc['Full_training_time'].to_numpy()) 
           
           # Model quality
           # accuracy
           test_acc = evaluate_score(df_full_cost_selection_vc['accuracy'].to_numpy(), df_full_cost_vc['accuracy'].to_numpy()) 
           # F1-score
           test_f1 = evaluate_score(df_full_result_selection_vc['F1_score'].to_numpy(), df_full_result_vc['F1_score'].to_numpy()) 
           #precision
           test_precision = evaluate_score(df_full_result_selection_vc['Precision'].to_numpy(), df_full_result_vc['Precision'].to_numpy()) 
           # recall 
           test_recall = evaluate_score(df_full_result_selection_vc['Recall'].to_numpy(), df_full_result_vc['Recall'].to_numpy()) 

           # Fairness
           # SPD gender
           test_spd_g = evaluate_score(df_full_result_selection_vc['SPD_gender'].to_numpy(), df_full_result_vc['SPD_gender'].to_numpy())
           # EOD gender
           test_eod_g = evaluate_score(df_full_result_selection_vc['EOD_gender'].to_numpy(), df_full_result_vc['EOD_gender'].to_numpy())
           # AOD gender
           test_aod_g = evaluate_score(df_full_result_selection_vc['AOD_gender'].to_numpy(), df_full_result_vc['AOD_gender'].to_numpy())
           # DI gender
           test_di_g = evaluate_score(df_full_result_selection_vc['DI_gender'].to_numpy(), df_full_result_vc['DI_gender'].to_numpy())
           # DcI gender
           test_dci_g = evaluate_score(df_full_result_selection_vc['DcI_gender'].to_numpy(), df_full_result_vc['DcI_gender'].to_numpy())

           # SPD age
           test_spd_a = evaluate_score(df_full_result_selection_vc['SPD_age'].to_numpy(), df_full_result_vc['SPD_age'].to_numpy())
           # EOD age
           test_eod_a = evaluate_score(df_full_result_selection_vc['EOD_age'].to_numpy(), df_full_result_vc['EOD_age'].to_numpy())
           # AOD age
           test_aod_a = evaluate_score(df_full_result_selection_vc['AOD_age'].to_numpy(), df_full_result_vc['AOD_age'].to_numpy())
           # DI age
           test_di_a = evaluate_score(df_full_result_selection_vc['DI_age'].to_numpy(), df_full_result_vc['DI_age'].to_numpy())
           # DcI age
           test_dci_a = evaluate_score(df_full_result_selection_vc['DcI_age'].to_numpy(), df_full_result_vc['DcI_age'].to_numpy())

        



           avg_sel_data = pd.DataFrame({
            **{str(col) + '_avg_avg': [result_sel_avg_avg[col]] for col in result_sel_avg_avg.index},
            **{str(col) + '_avg_avg': [cost_sel_avg_avg[col]] for col in cost_sel_avg_avg.index},
            **{str(col) + '_avg_avg': [cost_time_sel_avg_avg[col]] for col in cost_time_sel_avg_avg.index},
            **{str(col) + '_std_avg': [result_sel_std_avg[col]] for col in result_sel_std_avg.index},
            **{str(col) + '_std_avg': [cost_sel_std_avg[col]] for col in cost_sel_std_avg.index},
            **{str(col) + '_std_avg': [cost_time_sel_std_avg[col]] for col in cost_time_sel_std_avg.index},
            **{str(col) + '_vc_avg': [result_sel_vc_avg[col]] for col in result_sel_vc_avg.index},
            **{str(col) + '_vc_avg': [cost_sel_vc_avg[col]] for col in cost_sel_vc_avg.index},
            **{str(col) + '_vc_avg': [cost_time_sel_vc_avg[col]] for col in cost_time_sel_vc_avg.index},
           'dataset': ['mobiact'],
           'model': [model],
           'system': [system],
           'ratio': [directory_path_2.split('_')[-1]],
           'test_time' : [test_time], 
           'test_acc' : [test_acc], 
           'test_f1' : [test_f1], 
           'test_precision' : [test_precision], 
           'test_recall' : [test_recall],
           'test_SPD_gender' : [test_spd_g], 
           'test_EOD_gender' : [test_eod_g], 
           'test_AOD_gender' : [test_aod_g], 
           'test_DI_gender' : [test_di_g], 
           'test_DcI_gender' : [test_dci_g], 

           'test_SPD_age' : [test_spd_a],
           'test_EOD_age' : [test_eod_a],
           'test_AOD_age' : [test_aod_a], 
           'test_DI_age' : [test_di_a], 
           'test_DcI_age' : [test_dci_a], 

           'test_SPD_race' : ' ',
           'test_EOD_race' : ' ',
           'test_AOD_race' : ' ', 
           'test_DI_race' : ' ',
           'test_DcI_race' : ' ',
            })
           
           df_average_results = pd.concat([df_average_results, avg_sel_data], ignore_index=True)




 

df_average_results.to_csv('../results/test/mobiact_ttest_results_epochs_intervals_vc_metrics.csv', index=False)



Concat datasets' T-test results

In [ ]:
import pandas as pd

def concatenate_csv_files(file_paths):
    target_columns = [
        'accuracy_avg_avg', 'F1_score_avg_avg', 'Precision_avg_avg', 'Recall_avg_avg',
        'SPD_gender_avg_avg', 'EOD_gender_avg_avg', 'AOD_gender_avg_avg', 'DI_gender_avg_avg', 'DcI_gender_avg_avg',
        'SPD_age_avg_avg', 'EOD_age_avg_avg', 'AOD_age_avg_avg', 'DI_age_avg_avg', 'DcI_age_avg_avg',
        'SPD_race_avg_avg', 'EOD_race_avg_avg', 'AOD_race_avg_avg', 'DI_race_avg_avg', 'DcI_race_avg_avg',
        'accuracy_std_avg', 'F1_score_std_avg', 'Precision_std_avg', 'Recall_std_avg',
        'SPD_gender_std_avg', 'EOD_gender_std_avg', 'AOD_gender_std_avg', 'DI_gender_std_avg', 'DcI_gender_std_avg',
        'SPD_age_std_avg', 'EOD_age_std_avg', 'AOD_age_std_avg', 'DI_age_std_avg', 'DcI_age_std_avg',
        'SPD_race_std_avg', 'EOD_race_std_avg', 'AOD_race_std_avg', 'DI_race_std_avg', 'DcI_race_std_avg',
        'accuracy_vc_avg', 'F1_score_vc_avg', 'Precision_vc_avg', 'Recall_vc_avg',
        'SPD_gender_vc_avg', 'EOD_gender_vc_avg', 'AOD_gender_vc_avg', 'DI_gender_vc_avg', 'DcI_gender_vc_avg',
        'SPD_age_vc_avg', 'EOD_age_vc_avg', 'AOD_age_vc_avg', 'DI_age_vc_avg', 'DcI_age_vc_avg',
        'SPD_race_vc_avg', 'EOD_race_vc_avg', 'AOD_race_vc_avg', 'DI_race_vc_avg', 'DcI_race_vc_avg',
        'dataset', 'model', 'system', 'ratio', 'test_time', 'test_acc', 
        'test_f1', 'test_precision', 'test_recall', 'test_SPD_gender', 
        'test_EOD_gender', 'test_AOD_gender', 'test_DI_gender', 'test_DcI_gender', 
        'test_SPD_age', 'test_EOD_age', 'test_AOD_age', 'test_DI_age', 'test_DcI_age', 
        'test_SPD_race', 'test_EOD_race', 'test_AOD_race', 'test_DI_race', 'test_DcI_race'
    ]
    
    dfs = []

    # Read each CSV file
    for file_path in file_paths:
        df = pd.read_csv(file_path, usecols=lambda c: c in target_columns, engine='python')
        # Add missing columns with NaN values  
        missing_cols = set(target_columns) - set(df.columns)
        for col in missing_cols:
            df[col] = pd.NA
        # Order columns
        df = df.reindex(columns=target_columns)
        dfs.append(df)

    # Concatenate all DataFrames 
    result = pd.concat(dfs, ignore_index=True)

    return result

In [ ]:
file_paths = ['../results/test/ars_ttest_results_epochs_intervals_vc_metrics.csv', 
              '../results/test/dc_ttest_results_epochs_intervals_vc_metrics.csv', 
              '../results/test/mobiact_ttest_results_epochs_intervals_vc_metrics.csv', 
              '../results/test/census_ttest_results_epochs_intervals_vc_metrics.csv', 
              '../results/test/kdd_ttest_results_epochs_intervals_vc_metrics.csv']
result_df = concatenate_csv_files(file_paths)
result_df.to_csv('../results/test/std_avg_vc_ttest_results_with_vc.csv', index=False)

Post-processing T-test results 

Filtering T-test results based on variability difference, the impact is considered significant (positive or negative) if the difference between the full variability and the selection variability exceeds 1%

In [ ]:
import pandas as pd

def false_positives_negatives(path=None,thr_acc=0.01, thr=0.01, output_false_positives_vc=None, output_false_negatives_vc=None, output_false_positives_std=None, output_false_negatives_std=None, output=None):
    # Load the CSV file
    df = pd.read_csv(path)


    # Initialize DataFrames
    false_better_df = pd.DataFrame()
    false_worse_df = pd.DataFrame()

    # Columns to check for the value 'positive'
    columns_to_modify = [
        'test_acc', 'test_f1', 'test_precision', 'test_recall',
        'test_SPD_gender', 'test_EOD_gender', 'test_AOD_gender', 'test_DI_gender', 'test_DcI_gender', 
        'test_SPD_age', 'test_EOD_age', 'test_AOD_age', 'test_DI_age', 'test_DcI_age', 
        'test_SPD_race', 'test_EOD_race', 'test_AOD_race', 'test_DI_race', 'test_DcI_race'
    ]

    # Columns with the average metrics
    columns_to_check_std = ['accuracy_std_avg', 'F1_score_std_avg', 'Precision_std_avg', 'Recall_std_avg',
        'SPD_gender_std_avg', 'EOD_gender_std_avg', 'AOD_gender_std_avg', 'DI_gender_std_avg', 'DcI_gender_std_avg',
        'SPD_age_std_avg', 'EOD_age_std_avg', 'AOD_age_std_avg', 'DI_age_std_avg', 'DcI_age_std_avg',
        'SPD_race_std_avg', 'EOD_race_std_avg', 'AOD_race_std_avg', 'DI_race_std_avg', 'DcI_race_std_avg'   
    ]

    columns_to_check_vc = ['accuracy_vc_avg', 'F1_score_vc_avg', 'Precision_vc_avg', 'Recall_vc_avg',
        'SPD_gender_vc_avg', 'EOD_gender_vc_avg', 'AOD_gender_vc_avg', 'DI_gender_vc_avg', 'DcI_gender_vc_avg',
        'SPD_age_vc_avg', 'EOD_age_vc_avg', 'AOD_age_vc_avg', 'DI_age_vc_avg', 'DcI_age_vc_avg',
        'SPD_race_vc_avg', 'EOD_race_vc_avg', 'AOD_race_vc_avg', 'DI_race_vc_avg', 'DcI_race_vc_avg'   
    ]

    # Initialize references dictionary for vc 
    references = {col: None for col in columns_to_check_vc if col in df.columns}

    # Iterate through DataFrame
    for index, row in df.iterrows():
        if row['system'] == 'Full':
            references = {col: None for col in columns_to_check_vc if col in df.columns}
            # Set the reference values for metrics when system is 'Full'
            for col in columns_to_check_vc:
                if col in df.columns:
                    references[col] = row[col]
        else:
            # Check other rows where system is not 'Full'
            for test_col, avg_col in zip(columns_to_modify, columns_to_check_vc):
                if test_col in df.columns and avg_col in df.columns and row[test_col] == 'positive' :
                    threshold = thr  

                    if references.get(avg_col) is not None and (abs(references[avg_col] - row[avg_col]) < threshold):
                        df.at[index, test_col] = 'insignificant-p-vc'
                        if row['system'] != 'Random' and row['ratio'] != 0.5:
                            # Add row to 'false better' DataFrame if certain conditions are met
                            false_better_df = pd.concat([false_better_df, pd.DataFrame([row])], ignore_index=True)

                if test_col in df.columns and avg_col in df.columns and row[test_col] == 'negative':
                    threshold = thr  # Set threshold based on DI metrics

                    if references.get(avg_col) is not None and (abs(references[avg_col] - row[avg_col]) < threshold):
                        df.at[index, test_col] = 'insignificant-n-vc'
                        if row['system'] != 'Random' and row['ratio'] != 0.5:
                            # Add row to 'false better' DataFrame if certain conditions are met
                            false__worse_df = pd.concat([false_worse_df, pd.DataFrame([row])], ignore_index=True)

    #########################################  
    # Initialize DataFrames
    false_better_df = pd.DataFrame()
    false_worse_df = pd.DataFrame()  
    # Initialize references dictionary for vc 
    references = {col: None for col in columns_to_check_std if col in df.columns}

    # Iterate through DataFrame
    for index, row in df.iterrows():
        if row['system'] == 'Full':
            references = {col: None for col in columns_to_check_std if col in df.columns}
            # Set the reference values for metrics when system is 'Full'
            for col in columns_to_check_std:
                if col in df.columns:
                    references[col] = row[col]
        else:
            # Check other rows where system is not 'Full'
            for test_col, avg_col in zip(columns_to_modify, columns_to_check_std):
                if test_col in df.columns and avg_col in df.columns and row[test_col] == 'positive' :
                    threshold = thr 
                    

                    if references.get(avg_col) is not None and (abs(references[avg_col] - row[avg_col]) < threshold):
                        df.at[index, test_col] = 'insignificant-p-std'
                        if row['system'] != 'Random' and row['ratio'] != 0.5:
                            # Add row to 'false better' DataFrame if certain conditions are met
                            false_better_df = pd.concat([false_better_df, pd.DataFrame([row])], ignore_index=True)

                if test_col in df.columns and avg_col in df.columns and row[test_col] == 'negative':
                    threshold = thr  # Set threshold based on DI metrics

                    if references.get(avg_col) is not None and (abs(references[avg_col] - row[avg_col]) < threshold):
                        df.at[index, test_col] = 'insignificant-n-std'
                        if row['system'] != 'Random' and row['ratio'] != 0.5:
                            # Add row to 'false better' DataFrame if certain conditions are met
                            false__worse_df = pd.concat([false_worse_df, pd.DataFrame([row])], ignore_index=True)

    # Save 'false better' rows to a new CSV file
    false_better_df.to_csv(output_false_positives_std, index=False)
    false__worse_df.to_csv(output_false_negatives_std, index=False)

    # Save the modified DataFrame to a new CSV file
    df.to_csv(output, index=False)

    return len(false_better_df) + len(false__worse_df)



In [ ]:
path='../results/test/std_avg_vc_ttest_results_with_vc.csv'
thr=0.01
thr_acc=0.01
output_false_positives_vc='../results/test/false_positives_vc_vc_ttest_results_with_vc.csv'
output_false_negatives_vc='../results/test/false_negatives_vc_vc_ttest_results_with_vc.csv'
output_false_positives_std='../results/test/false_positives_vc_std_ttest_results_with_vc.csv'
output_false_negatives_std='../results/test/false_negatives_vc_std_ttest_results_with_vc.csv'
output='../results/test/std_avg_vc_ttest_results_with_vc_4c.csv'

false_positives_negatives(path, thr_acc, thr, output_false_positives_vc, output_false_negatives_vc, output_false_positives_std, output_false_negatives_std, output)

Focus on cases where model accuracy is severely degraded, in such cases, variability impact is considered insignificant.

In [ ]:
import pandas as pd


def false_positives_negatives_acc(path= '../results/test/std_avg_vc_ttest_results_with_vc_4c.csv', x = 10):
    # Initialize the reference accuracy
    ref_accuracy = None
    # DataFrame to hold rows that are 'false better'
    false_better_df = pd.DataFrame()

    df = pd.read_csv(path) 

    # Columns to check for the value 'positive'
    columns_to_modify = ['test_SPD_gender', 'test_EOD_gender', 'test_AOD_gender', 'test_DI_gender', 'test_DcI_gender', 
                        'test_SPD_age', 'test_EOD_age', 'test_AOD_age', 'test_DI_age', 'test_DcI_age', 
                        'test_SPD_race', 'test_EOD_race', 'test_AOD_race', 'test_DI_race', 'test_DcI_race']

    for index, row in df.iterrows():
        if row['system'] == 'Full':
            ref_accuracy = row['accuracy_avg_avg']  # Set the reference accuracy
        elif ref_accuracy is not None:  # Ensure there is a reference to compare against
            current_accuracy = row['accuracy_avg_avg']
            if current_accuracy <= ref_accuracy - x  and row['ratio'] != 0.5:
                # Add row to DataFrame using pd.concat
                false_better_df = pd.concat([false_better_df, pd.DataFrame([row])], ignore_index=True)
                # Modify specific columns if their values are 'positive'
                for col in columns_to_modify:
                    if row[col] == 'positive' or row[col] == 'negative':
                        df.at[index, col] = 'insignificant-acc-vc'


    # Save 'false better' rows to a new CSV file
    path_o= '../results/test/false_positives_negatives_' + str(x) + '_' + path.split('/')[1]
    false_better_df.to_csv(path_o, index=False)


    # Save the modified DataFrame to a new CSV file
    path_m= '../results/test/std_avg_vc_ttest_results_with_vc_5c.csv'
    df.to_csv(path_m, index=False)

    return len(false_better_df)


In [ ]:
false_positives_negatives_acc('../results/test/std_avg_vc_ttest_results_with_vc_4c.csv', 10)

Post-process outputs 

In [ ]:
import pandas as pd

def concat_filter_csv(file_path, output):

    # Concaténation des DataFrames
    result_df = pd.read_csv(file_path)

    # Remplacement de 'significant-p' par 'significant'

    result_df.replace('insignificant-acc-vc', 'insignificant', inplace=True)

    result_df.replace('insignificant-n-std', 'insignificant', inplace=True)

    result_df.replace('insignificant-p-std', 'insignificant', inplace=True)

    result_df.replace('insignificant-n-vc', 'insignificant', inplace=True)

    result_df.replace('insignificant-p-vc', 'insignificant', inplace=True)

    result_df.to_csv(output + '-wo-insignificant-vc.csv', index=False)

    # Filtrage des lignes où 'ratio' est '0.5' ou 'system' est 'Random'
    result_df = result_df[(result_df['ratio'] != 0.5) & (result_df['system'] != 'Full')]
    result_df.to_csv(output+'-w-random.csv', index=False)

    result_df = result_df[(result_df['system'] != 'Random')]
    result_df.to_csv(output+'-wo-random.csv', index=False)

    return result_df


In [ ]:
concat_filter_csv('../results/test/std_avg_vc_ttest_results_with_vc_5c.csv', '../results/test/std_avg_vc_ttest_results_with_vc_5c')